In [1]:
import os

list_file = sorted(list(os.walk('./file/3.part_code'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_2.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/3.part_code/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['Product No'] = input_['Product No'].astype(int)
input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Code'].tolist()

总数量：6258


['4219-6772',
 '0094-10815',
 '7316-00903',
 '4656-0732',
 '4656-0733',
 '4656-0734',
 '4656-0751',
 '4656-0830',
 '4656-0831',
 '4656-0832',
 '4656-0833',
 '7743-00525',
 '7743-00538',
 '7743-00540',
 '7743-00545',
 '7743-00550',
 '7743-00555',
 '7743-00560',
 '7743-01019',
 '7743-01022',
 '7743-01025',
 '7743-01035',
 '7743-01038',
 '7743-01040',
 '7743-01045',
 '7743-01050',
 '7743-01052',
 '7743-01055',
 '7743-01060',
 '7743-01352',
 '4230-04376',
 '4230-04377',
 '4230-04378',
 '4230-04379',
 '4281-17123',
 '4281-17126',
 '4281-17127',
 '4281-17129',
 '4281-17130',
 '4281-17132',
 '4281-17133',
 '4281-17134',
 '4281-17135',
 '4281-17136',
 '4281-17137',
 '4281-17138',
 '4281-17141',
 '4281-17142',
 '4281-17143',
 '4278-18008',
 '4278-18015',
 '4278-18018',
 '4278-18022',
 '4278-18028',
 '4278-18030',
 '4278-18032',
 '4278-18038',
 '4278-18045',
 '4278-18050',
 '4278-18055',
 '4278-18060',
 '4278-18065',
 '4278-18070',
 '4278-18075',
 '4278-18080',
 '4281-18106',
 '4281-18108',
 '42

In [5]:
a = 1

a

1

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

if input_.loc[a, 'Country'] == 'US':
    country, lang = 'us', 'en'
elif input_.loc[a, 'Country'] == 'DE':
    country, lang = 'de', 'de'
elif input_.loc[a, 'Country'] == 'UK':
    country, lang = 'gb', 'en'
else:
    country, lang = 'fuck', 'fuck'

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = f'''https://www.gates.com/{country}/{lang}/search.p.{input_.loc[a, 'Product Number']}.v.{input_.loc[a, 'Part Code']}.html'''
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            if html.xpath('//div[@class="product-tabs__fixed"]/ul[@role="tablist"]'):
                with open('./part.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
import json

dict_ = json.loads(resp.text.split('new s7viewers.MixedMediaViewer(')[1].split('"localizedTexts": localisedTexts')[0].strip().removesuffix(',') + '}')

dict_

{'containerId': 's7mixedmedia_div',
 'params': {'serverurl': 'https://gates.scene7.com/is/image/',
  'contenturl': 'https://gates.scene7.com/is/content/',
  'config': 'gates/InlineMixedMedia_light',
  'videoserverurl': 'https://gates.scene7.com/is/content',
  'asset': 'gates/image-coming-soon'}}

In [9]:
list_th = [f'''Kit_{text.strip().removesuffix('#').strip().title()}''' for text in html.xpath('//div[@id="product-kit-content__content"]/div[@class="accordion-content__wrapper"]/table[@class="table c-table c-table--striped"]/thead/tr/th/text()')]

list_th

[]

In [10]:
list_tr = html.xpath('//div[@id="product-kit-content__content"]/div[@class="accordion-content__wrapper"]/table[@class="table c-table c-table--striped"]/tbody/tr')

list_tr

[]

In [11]:
dict_kit = {}
for i, tr in enumerate(list_tr):
    dict_kit[str(i)], list_td = {}, tr.xpath('./td')
    for j, td in enumerate(list_td):
        dict_kit[str(i)][list_th[j]] = td.xpath('./text()')[0].strip() if td.xpath('./text()') else ''

dict_kit

{}

In [12]:
list_tr = html.xpath('//li[@class="nested-accordion__item nested-accordion__item--level1"]/div[@role="tabpanel"]/table[@class="table c-table c-table--split"]/tbody/tr')

list_tr

[<Element tr at 0x11e7a5bc0>,
 <Element tr at 0x11fb8e800>,
 <Element tr at 0x11fb8d740>]

In [13]:
i, part_number, sku, title, upc, dict_information = 0, '', '', '', '', {}
if input_.loc[a, 'Country'] == 'DE':
    for tr in list_tr:
        key, value = tr.xpath('./th/text()')[0].strip().removesuffix('#').strip(), tr.xpath('./td/text()')[0].strip() if tr.xpath('./td/text()') else ''
        if key == 'Teilenummer':
            part_number = value
        elif key == 'Produktnummer':
            sku = value
        elif key == 'Beschreibung':
            title = value
        elif key == 'UPC':
            upc = value
        else:
            dict_information[str(i)] = {key: value}
            i += 1
else:
    for tr in list_tr:
        key, value = tr.xpath('./th/text()')[0].strip().removesuffix('#').strip(), tr.xpath('./td/text()')[0].strip() if tr.xpath('./td/text()') else ''
        if key == 'Part':
            part_number = value
        elif key == 'Product':
            sku = value
        elif key == 'Description':
            title = value
        elif key == 'UPC':
            upc = value
        else:
            dict_information[str(i)] = {key: value}
            i += 1

part_number, sku, title, upc, dict_information

('',
 '0094-10815',
 '6REEL FL MERCHANDISER W/O HOSE',
 '',
 {'0': {'Gewicht (kg)': ''}})

In [14]:
df_temp = pd.DataFrame([{'Product No': input_.loc[a, 'Product No'],
                         'Product Name': input_.loc[a, 'Product Name'],
                         'Product Number': input_.loc[a, 'Product Number'],
                         'Country': input_.loc[a, 'Country'],
                         'No': input_.loc[a, 'No'],
                         'Part Number': part_number,
                         'Type': input_.loc[a, 'Type'],
                         'SKU': sku,
                         'UPC': upc,
                         'Title': title,
                         'Url': request_url,
                         'Json_Information': json.dumps(dict_information),
                         'Json_Kit': json.dumps(dict_kit),
                         'Part Code': input_.loc[a, 'Part Code'],
                         'Asset': dict_['params']['asset'].strip()}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Product No,Product Name,Product Number,Country,No,Part Number,Type,SKU,UPC,Title,Url,Json_Information,Json_Kit,Part Code,Asset
0,2,6 Reel,0094-000000-000003,DE,1,,HoseVariantProduct,0094-10815,,6REEL FL MERCHANDISER W/O HOSE,https://www.gates.com/de/de/search.p.0094-0000...,"{""0"": {""Gewicht (kg)"": """"}}",{},0094-10815,gates/image-coming-soon


In [15]:
crawler_status = 'ok'

crawler_status

'ok'

In [16]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
output_error

""


In [17]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Product No', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,Product No,Product Name,Product Number,Country,No,Part Number,Type,SKU,UPC,Title,Url,Json_Information,Json_Kit,Part Code,Asset
0,2,6 Reel,0094-000000-000003,DE,1,,HoseVariantProduct,0094-10815,,6REEL FL MERCHANDISER W/O HOSE,https://www.gates.com/de/de/search.p.0094-0000...,"{""0"": {""Gewicht (kg)"": """"}}",{},0094-10815,gates/image-coming-soon


In [18]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Product No', 'No'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel('./test_part.xlsx', index=False)

output_error

""
